In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.pipeline import FeatureUnion
from nltk.corpus import stopwords 
import os
import string

import heapq 


def removeStopWords(wordList):
    return [word for word in wordList if word not in stopwords.words('english')]

# DO NOT USE
def calcCosSimOld(aMat,bVec):
    # Calculate cosine simularity
    a = np.sum(aMat,axis=0)/aMat.shape[0]
    b = bVec.copy()
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

def calcCosSim(a,b):
    # Calculate cosine simularity
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

def removeStem(sentence):
    ps = PorterStemmer()
    words = word_tokenize(sentence)
    tmpStr = ''
    for w in words:
        tmpStr += ps.stem(w) + ' '
    return tmpStr


In [5]:
rootPath = './hotel/'
complaintSubPath = 'hotelMain/'

contractPath = './hotel_policy.txt.'
#contractPath = rootPath + '/termService/' + 'related.txt'
#contractPath = rootPath + '/termService/' + 'unrelated.txt'

contractTxt = open(contractPath, encoding="utf8", errors='ignore').read()
contractVec = removeStopWords([removeStem(contractTxt)])

vec = CountVectorizer()
contractFreq = vec.fit_transform(contractVec)


compList = []

# Used for removing stem from words
for cFilePath in os.listdir(rootPath + complaintSubPath):
    cFilePath = rootPath + complaintSubPath + cFilePath
    cFileTxt = open(cFilePath,'r').read()

    cFileTxt = removeStem(cFileTxt)
    
    compList.append(cFileTxt + ' ')
    

compList = [''.join(str(v) for v in compList)]  # We are combining all complaints into one
complantCnt = CountVectorizer()
a = complantCnt.fit_transform(compList)
#print(contractVec)

In [7]:
resPara = []
resScore = []

splitNum = 300
#for paragraph in contractTxt.split('\n'):
for paragraph in [contractTxt[i:i+splitNum] for i in range(0, len(contractTxt), splitNum)]:
    paragraph = paragraph.translate(string.punctuation)


    try:
        vec = CountVectorizer()
        contractFreq = vec.fit_transform([paragraph])
    except:
        continue
    complaintDf = pd.DataFrame(a.toarray(),columns=complantCnt.get_feature_names())
    contractDf = pd.DataFrame(contractFreq.toarray(),columns=vec.get_feature_names())

    combinedDf = pd.concat([complaintDf, contractDf],sort=False).fillna(value=0.0)
    complainVec = combinedDf.iloc[0].values
    contractVec = combinedDf.iloc[1].values

    simRes = calcCosSim(complainVec,contractVec)
    #print(simRes)
    #print('---------------------')
    resPara.append(paragraph)
    resScore.append(simRes)



#print(resPara[np.argmax(resScore)])
#print('')
#print(np.max(resScore))



rankAr = np.asarray(resPara).argsort()[::-1][:5]

heapLi = []


print('Total number of parts in contract: ', len(contractTxt.split('\n')))
      
print('\n\n')
for ind in rankAr:    
    heapq.heappush(heapLi,[-resScore[ind],ind])
    
while len(heapLi) > 0:
    score,ind = heapq.heappop(heapLi)
    score = score*-1
    
    print(resPara[ind])
    print('')
    print(score)
    print('--------------------------\n')


Total number of parts in contract:  488



you will be charged the amount in the currency that you select immediately. If you select to pay later at the hotel itself, the hotel will charge you in the local currency of the relevant hotel at the time of your stay. Tax rates and foreign exchange rates could change in the time between booking an

0.5724030359360835
--------------------------

you book online so we can add the nights you collect to your account after your stay. If you make a booking with us by phone, you must tell us the email address on your account so we know where to add the nights. Only Hotels.com Rewards members collect nights. Other guests on the same booking do not

0.43601271011287446
--------------------------

whatsoever with respect to the Mobile Application;+5. you acknowledge that we, not Apple, are responsible for addressing any claims of yours or any third party relating to the Mobile Application;+6. you acknowledge that, in the event of any third party claim